# Data Streaming for Multiple Tickers

This section demonstrates how to stream real-time market data for multiple financial instruments simultaneously. This is useful for monitoring portfolios, arbitrage opportunities, or building real-time dashboards.

In [1]:
import pandas as pd
from ib_async import *
import time
util.startLoop()

In [2]:
ib = IB()
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

## Setup Multiple Contracts

First, let's define multiple contracts we want to stream data for:

In [3]:
# Define contracts for different asset types
contracts = [
    Forex("EURUSD"),    # EUR/USD forex pair
    Forex("GBPUSD"),    # GBP/USD forex pair  
    Forex("USDJPY"),    # USD/JPY forex pair
]

# Qualify all contracts
qualified_contracts = []
for contract in contracts:
    qualified = ib.qualifyContracts(contract)
    if qualified:
        qualified_contracts.append(qualified[0])

print(f"Successfully qualified {len(qualified_contracts)} contracts:")
for contract in qualified_contracts:
    print(f"  - {contract.symbol}: {contract}")

Successfully qualified 3 contracts:
  - EUR: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
  - GBP: Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
  - USD: Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')


## Start Market Data Streams

Now let's request market data for all contracts and store the ticker objects:

In [4]:
# Request market data for all contracts
tickers = []
for contract in qualified_contracts:
    ticker = ib.reqMktData(contract)
    tickers.append(ticker)

print(f"Started streaming data for {len(tickers)} instruments")

# Wait a moment for data to start flowing
ib.sleep(2)

# Display initial data
for i, ticker in enumerate(tickers):
    print(f"{ticker.contract.symbol}: Bid={ticker.bid}, Ask={ticker.ask}, Last={ticker.last}")

Started streaming data for 3 instruments
EUR: Bid=1.15717, Ask=1.15718, Last=1.15715
GBP: Bid=1.31912, Ask=1.31916, Last=1.31915
USD: Bid=156.039, Ask=156.043, Last=156.04


## Real-Time Data Streaming Loop

Now let's create a loop that continuously displays updated market data for all our instruments. This demonstrates how you might monitor multiple assets in real-time:

In [5]:
# Debug: Check what we actually have
print("Available tickers:")
for ticker in tickers:
    print(f"  - Symbol: {ticker.contract.symbol}, Contract: {ticker.contract}")

# Create mapping for easier access
ticker_map = {ticker.contract.symbol: ticker for ticker in tickers}
print(f"\nTicker map keys: {list(ticker_map.keys())}")

# Check if we have the expected symbols
required_symbols = ['EURUSD', 'GBPUSD', 'USDJPY']
available_symbols = list(ticker_map.keys())

if not all(symbol in available_symbols for symbol in required_symbols):
    print(f"Warning: Not all required symbols available.")
    print(f"Required: {required_symbols}")
    print(f"Available: {available_symbols}")
    
    # Use whatever symbols we have
    if len(available_symbols) >= 3:
        data1 = tickers[0]  # First available
        data2 = tickers[1]  # Second available  
        data3 = tickers[2]  # Third available
        print(f"Using: {data1.contract.symbol}, {data2.contract.symbol}, {data3.contract.symbol}")
    else:
        print("Not enough tickers available. Please check connection and contracts.")
        # Exit early if not enough data
else:
    # Get references to individual tickers for the loop
    data1 = ticker_map['EURUSD']  # EUR/USD
    data2 = ticker_map['GBPUSD']  # GBP/USD  
    data3 = ticker_map['USDJPY']  # USD/JPY

if len(tickers) >= 3:
    print("Starting real-time data stream...")
    print("Symbol       | Market Price 1 | Market Price 2 | Market Price 3")
    print("-" * 60)

    # Stream data for 10 iterations (similar to the screenshot example)
    for i in range(10):
        price1 = round(data1.marketPrice(), 5) if data1.marketPrice() else 'N/A'
        price2 = round(data2.marketPrice(), 3) if data2.marketPrice() else 'N/A'
        price3 = round(data3.marketPrice(), 3) if data3.marketPrice() else 'N/A'
        print(f"Update {i+1:2d}    | {price1:>13} | {price2:>13} | {price3:>13}")
        ib.sleep(1)
else:
    print("Cannot start streaming - insufficient ticker data")

Available tickers:
  - Symbol: EUR, Contract: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')
  - Symbol: GBP, Contract: Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD')
  - Symbol: USD, Contract: Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY')

Ticker map keys: ['EUR', 'GBP', 'USD']
Required: ['EURUSD', 'GBPUSD', 'USDJPY']
Available: ['EUR', 'GBP', 'USD']
Using: EUR, GBP, USD
Starting real-time data stream...
Symbol       | Market Price 1 | Market Price 2 | Market Price 3
------------------------------------------------------------
Update  1    |       1.15719 |         1.319 |        156.04
Update  2    |       1.15717 |         1.319 |        156.04
Update  3    |       1.15717 |         1.319 |        156.04
Update  4    |       1.15717 |         1.319 |        156.04
Update  5    |       1.15717 |         1.319 |        156.04
U

## Alternative: Simple Loop Format

A simple loop that prints market prices:

In [6]:
for i in range(10):
    print(round(data1.marketPrice(), 5), round(data2.marketPrice(), 3), round(data3.marketPrice(), 3))
    ib.sleep(1)

1.15719 1.319 156.04
1.15719 1.319 156.04
1.15719 1.319 156.04
1.15719 1.319 156.04
1.15719 1.319 156.04
1.15719 1.319 156.04
1.15719 1.319 156.037
1.15719 1.319 156.037
1.15719 1.319 156.035
1.15719 1.319 156.035


## Creating a Data Streaming DataFrame

For more advanced analysis, you might want to collect the streaming data into a pandas DataFrame:

In [7]:
# Collect data into a list for DataFrame creation
data_records = []

print("Collecting data for DataFrame...")
for i in range(5):
    timestamp = pd.Timestamp.now()
    record = {
        'timestamp': timestamp,
        'EURUSD': data1.marketPrice(),
        'GBPUSD': data2.marketPrice(), 
        'USDJPY': data3.marketPrice()
    }
    data_records.append(record)
    print(f"Sample {i+1}: {record}")
    ib.sleep(1)

# Create DataFrame
df = pd.DataFrame(data_records)
df.set_index('timestamp', inplace=True)
print("\nCollected data as DataFrame:")
print(df)

Sample 1: {'timestamp': Timestamp('2025-11-25 20:24:36.195703'), 'EURUSD': 1.157195, 'GBPUSD': 1.31897, 'USDJPY': 156.04}
Sample 2: {'timestamp': Timestamp('2025-11-25 20:24:37.198022'), 'EURUSD': 1.157195, 'GBPUSD': 1.31897, 'USDJPY': 156.0375}
Sample 3: {'timestamp': Timestamp('2025-11-25 20:24:38.199391'), 'EURUSD': 1.157195, 'GBPUSD': 1.31897, 'USDJPY': 156.0375}
Sample 4: {'timestamp': Timestamp('2025-11-25 20:24:39.201401'), 'EURUSD': 1.15719, 'GBPUSD': 1.31895, 'USDJPY': 156.0375}
Sample 5: {'timestamp': Timestamp('2025-11-25 20:24:40.202866'), 'EURUSD': 1.15719, 'GBPUSD': 1.31895, 'USDJPY': 156.0375}

Collected data as DataFrame:
                              EURUSD   GBPUSD    USDJPY
timestamp                                              
2025-11-25 20:24:36.195703  1.157195  1.31897  156.0400
2025-11-25 20:24:37.198022  1.157195  1.31897  156.0375
2025-11-25 20:24:38.199391  1.157195  1.31897  156.0375
2025-11-25 20:24:39.201401  1.157190  1.31895  156.0375
2025-11-25 20:24:4

## Clean Up - Cancel Market Data

When you're done streaming, it's important to cancel the market data subscriptions to free up resources:

In [8]:
# Cancel market data subscriptions
for ticker in tickers:
    ib.cancelMktData(ticker.contract)
    print(f"Cancelled market data for {ticker.contract.symbol}")

print("All market data subscriptions cancelled.")

# Disconnect from IB
ib.disconnect()

Cancelled market data for EUR
Cancelled market data for GBP
Cancelled market data for USD
All market data subscriptions cancelled.


'Disconnecting from 127.0.0.1:7497, 527 B sent in 17 messages, 49.2 kB received in 1382 messages, session time 60.8 s.'